# Chapter 3 - Classification
The following notebook consists of different code examples for graphs seen in Chapter 3.

In [2]:
import sys
sys.path.append("../")
from utils import *

## Model Evaluations

In [ ]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

models = [LogisticRegression(), SVC(kernel='rbf', probability=True), KNeighborsClassifier(n_neighbors=4)]
model_names = [r"$\text{Logistic Regression}$", r"$\text{RBF SVM}$", r"$k-\text{NN}$"]

from sklearn.datasets import make_moons
X, y = make_moons(n_samples=500, noise=0.2, random_state=1)

In [ ]:
lims = np.array([X.min(axis=0), X.max(axis=0)]).T + np.array([-.2, .2])

fig = make_subplots(rows=1, cols=3, subplot_titles=model_names, horizontal_spacing = 0.01)
for i, m in enumerate(models):
    fig.add_traces([decision_surface(m.fit(X, y).predict, lims[0], lims[1], showscale=False),
                    go.Scatter(x=X[:,0], y=X[:,1], mode="markers", showlegend=False,
                               marker=dict(color=y, symbol=class_symbols[y], colorscale=class_colors(3), 
                                           line=dict(color="black", width=1)))], 
                   rows=1, cols=i+1)

fig.update_layout(width=1000, height=300).update_xaxes(visible=False).update_yaxes(visible=False)
fig.write_image(f"../figures/decision_boundary.png")
fig.show()

In [ ]:
from sklearn import metrics

fig = go.Figure(layout=go.Layout(title=r"$\text{ROC Curves Of Models - Gaussians Dataset}$", 
                                 xaxis=dict(title=r"$\text{False Positive Rate (FPR)}$"),
                                 yaxis=dict(title=r"$\text{True Positive Rate (TPR)}$")),
                data=[go.Scatter(x=[0,1], y=[0,1], mode="lines", showlegend=False, line_color="black", line_dash='dash')])

for i, model in enumerate(models):
    fpr, tpr, th = metrics.roc_curve(y, model.predict_proba(X)[:, 1])
    fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=model_names[i]))

    
fig.update_layout(width=800, height=500, yaxis=dict(range=[0,1.1]))
fig.write_image("../figures/roc.png")
fig.show()

## Perceptron

In [89]:
from sklearn.datasets import make_classification, make_blobs
from sklearn.linear_model import Perceptron as Perceptron

np.random.seed(9)

def create_linearly_separable_dataset(n=30, scale=10):
    # Create a dataset that is linearly separable
    separable = False
    while not separable:
        samples = make_classification(n_samples=n, n_features=2, n_informative=2, n_repeated = 0, 
                                  n_redundant=0,n_clusters_per_class=1, class_sep = 0.5, scale=scale)
        red, blue = samples[0][samples[1] == 0], samples[0][samples[1] == 1]
        separable = any([red[:, k].max() < blue[:, k].min() or red[:, k].min() > blue[:, k].max() for k in range(2)])
        
    X, y = samples[0], samples[1]
    
    # Make sure that the first two samples are from both classes
    idx = np.array([np.where(y == 0)[0][0], np.where(y == 1)[0][0]])
    msk = np.array([False]*X.shape[0])
    msk[idx] = True
    return np.r_[X[msk], X[~msk]], np.r_[y[msk], y[~msk]]

X, y = create_linearly_separable_dataset(16)
lim = np.array([X.min(axis=0), X.max(axis=0)]).T + np.array([-.5,.5])

frames = []
per = Perceptron()
per.partial_fit([X[0]], [y[0]], np.unique(y))

rnd = 1
while per.score(X, y) != 1:
    for i in range(X.shape[0]):
        # Perform another fitting over new sample
        per.partial_fit([X[i]], [y[i]], np.unique(y))

        # Get Perceptron separator
        w = per.coef_[0]
        yy = (-w[0] / w[1]) * lim[0] - (per.intercept_[0] / w[1])

        # Create animation frame
        frames.append(go.Frame(
            data = [
                go.Scatter(x = X[:,0], y=X[:, 1], mode = 'markers', showlegend=False,
                           marker = dict(size = 10, color = y, line=dict(color="black", width=1),
                                         symbol=class_symbols[y], colorscale=class_colors(2), opacity = [1]*(i+1) + [0.2]*(X.shape[0] - i-1))),
                go.Scatter(x = lim[0], y = [yy[0], yy[1]], mode = 'lines', line_color="black", showlegend=False)],
            traces=[0, 1],
            layout = go.Layout(title=rf"$\text{{Perceptron Fit - Round {rnd} After {i+1} Samples}}$")))
    rnd += 1


fig = go.Figure(data=frames[0]["data"],
                frames=frames,
                layout = go.Layout(
                    title=frames[0]["layout"]["title"],
                    xaxis=dict(range=lim[0], autorange=False),
                    yaxis=dict(range=lim[1], autorange=False),
                    updatemenus=[dict(type="buttons", buttons=[AnimationButtons.play(frame_duration=100), AnimationButtons.pause()])]))


animation_to_gif(fig, "../figures/perceptron_fit.gif", 500, width=700, height=700) 
fig.show()

## Decision Trees As Function Of Depth

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

np.random.seed(7)

# Sample data generated by a decision tree of depth 6
true_depth = 6
X, y = create_data_bagging_utils(n_samples = 1000, d = true_depth)
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Over specified depth, fit a decision tree and calculate train- and test errors
depths = np.array(range(1, 30))
frames, train_error, test_error = [], np.zeros(len(depths)), np.zeros(len(depths))
for k, depth_tree in enumerate(depths):
    # Fit model
    clf = DecisionTreeClassifier(max_depth=depth_tree, random_state=42).fit(X_train, y_train)
    
    # Evaluate over data sets
    train_error[k] = (1 - np.mean(clf.predict(X_train) == y_train))
    test_error[k]  = (1 - np.mean(clf.predict(X_test)  == y_test))

    # Create animation frame
    frames.append(go.Frame(
        data=[decision_surface(clf.predict, [X[:,0].min(), X[:,0].max()], [X[:,1].min(), X[:,1].max()], showscale = False),
              go.Scatter(x=X_test[:,0], y=X_test[:,1], mode='markers', showlegend=False,
                         marker=dict(color = y_test, symbol=class_symbols[y_test], colorscale=class_colors(2))),
              go.Scatter(x=depths[:depth_tree], y=train_error[:depth_tree], name="Train Error", xaxis="x2", yaxis="y2"),
              go.Scatter(x=depths[:depth_tree], y=test_error[:depth_tree], name="Test Error", xaxis="x2", yaxis="y2")],
        layout=go.Layout(title=rf"$\text{{Fitting Decition Tree As Function Of Depth: }}k={depth_tree}\text{{ - True Model Depth {true_depth}}}$"),
        traces=[0,1,2,3]))
    
    
fig = make_subplots(
    rows=1, cols=2, subplot_titles=(r"$\text{Decisions Boundaries}$", r"$\text{Misclassification Errors}$"),
    horizontal_spacing=0.1)

fig = fig.add_traces(data=frames[0]["data"], rows=[1, 1, 1, 1], cols=[1, 1, 2, 2])\
         .update(frames=frames)\
         .update_layout(updatemenus = [dict(type="buttons", buttons=[AnimationButtons.play(), AnimationButtons.pause()])],
                        width=1000, height=500, margin=dict(t=80), 
                        title=frames[0]["layout"]["title"])

# Update axes of left graph
fig.update_yaxes(title_text=r"$\text{Error Rate}$", range=[-0.05, 0.6], row=1, col=2)
fig.update_xaxes(title_text=r"$\text{Tree Depth}$", range=[depths[0], depths[-1]], row=1, col=2)

# Update axes of right graph
fig.update_yaxes(range=[0, 1], row=1, col=1)
fig.update_xaxes(range=[0, 1], row=1, col=1)

animation_to_gif(fig, "../figures/fit_decition_tree.gif", 1000, width=1000, height=500) 
fig.show()

## Generative Models - Simulation and Decision Boundaries

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# Generate dataset according to LDA generative model
mu = np.array([[0, 0], [2.5, 5], [5, 0]])
cov = np.array([[.5, 0], [0, .5]])
pi = [.33, .34, .33]

# y = np.random.binomial(n=1, p=pi, size=500)
y = np.random.choice([0,1,2], size=500, p=pi)
X = np.array([np.random.multivariate_normal(mu[yi], cov) for yi in y])


# Plotting dataset and LAD decision boundaries
lims = np.array([X.min(axis=0), X.max(axis=0)]).T + np.array([-.5, .5])


fig = go.Figure([decision_surface(LDA().fit(X, y).predict, lims[0], lims[1], showscale=False, 
                                  colorscale=class_colors(3), density=300),
                 go.Scatter(x=X[:,0], y=X[:,1], mode="markers", showlegend=False, 
                            marker=dict(color=y, symbol=class_symbols[y], colorscale=class_colors(3),
                                        line=dict(color="black", width=1)))],
                layout=go.Layout(xaxis=dict(range=lims[0], autorange=False),
                                 yaxis=dict(range=lims[1], autorange=False),
                                 width=600, height=400,
                                 title=r"$\text{Multi-class LDA Decision Boundary}$"))
fig.write_image("../figures/lda_decision_boundary.png")
fig.show()

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA

# Generate dataset according to QDA generative model
mu = np.array([[0, 0], [2.5, 5], [5, 0]])
cov = np.array([[[.5, 0], [0, .5]], [[.5, 0], [0, .5]], [[1, 0], [0, .5]],])
pi = [.33, .34, .33]

y = np.random.choice([0,1,2], size=500, p=pi)
X = np.array([np.random.multivariate_normal(mu[yi], cov[yi]) for yi in y])


# Plotting dataset and LAD decision boundaries
lims = np.array([X.min(axis=0), X.max(axis=0)]).T + np.array([-.5, .5])
fig = go.Figure([decision_surface(QDA().fit(X, y).predict, lims[0], lims[1], showscale=False, 
                                  colorscale=class_colors(3), density=300),
                 go.Scatter(x=X[:,0], y=X[:,1], mode="markers", showlegend=False, 
                            marker=dict(color=y, symbol=class_symbols[y], colorscale=class_colors(3),
                                        line=dict(color="black", width=1)))],
                layout=go.Layout(xaxis=dict(range=lims[0], autorange=False),
                                 yaxis=dict(range=lims[1], autorange=False),
                                 width=600, height=400,
                                 title=r"$\text{Multi-class QDA Decision Boundary}$"))
fig.write_image("../figures/qda_decision_boundary.png")
fig.show()

## Adaboost 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import iplot

class StagedAdaBoostClassifier(AdaBoostClassifier):
    def __init__(self, **kwargs):
        super().__init__(*kwargs)
        self.sample_weights = []
        self.res_list = []

    def _boost(self, iboost, X, y, sample_weight, random_state):
        self.sample_weights.append(sample_weight.copy())
        self.res_list.append(super()._boost(iboost, X, y, sample_weight, random_state))
        return self.res_list[-1]

    def _iteration_callback(self, iboost, X, y, sample_weight, estimator_weight = None, estimator_error = None):
        self.sample_weights.append(sample_weight.copy())

# Construct dataset
from sklearn.datasets import make_gaussian_quantiles

X1, y1 = make_gaussian_quantiles(cov=2.,
                                 n_samples=20, n_features=2,
                                 n_classes=2, random_state=1)
X2, y2 = make_gaussian_quantiles(mean=(3, 3), cov=1.5,
                                 n_samples=30, n_features=2,
                                 n_classes=2, random_state=1)
X = np.concatenate((X1, X2))
y = np.concatenate((y1, - y2 + 1))
x_range = (np.arange(min(X[:,0]), max(X[:,0]), 0.2)).tolist()
y_range = (np.arange(min(X[:,1]), max(X[:,1]), 0.2)).tolist()
xx, yy = np.meshgrid(x_range,y_range)
xx = xx.ravel()
yy = yy.ravel()

model = StagedAdaBoostClassifier().fit(X, y)
ensemble_predictions_staged = list(model.staged_score(X, y)) # For line error graph
staged_meshgrid_predictions = np.array(list(model.staged_decision_function(np.vstack([xx, yy]).T))) # For decisions boudaries

frames = []
for i in range(len(staged_meshgrid_predictions)):
    frames.append(go.Frame(
               data=[go.Scatter(x= X[:,0],  y= X[:,1], mode='markers', marker = dict(color =  y, size = np.maximum(200*model.sample_weights[i]+1, np.ones(len(model.sample_weights[i]))*3))), 
                     go.Scatter(x= np.arange(len(ensemble_predictions_staged))[:i],  y= ensemble_predictions_staged[:i], mode='lines+markers'),
                     go.Scatter(x= xx,  y= yy, mode='markers',  marker = dict(symbol = "square",
                                                                             color = staged_meshgrid_predictions[i,:]), opacity = 0.4, showlegend=False),
                     go.Scatter(x= X[:,0],  y= X[:,1], mode='markers', marker = dict(color = y)),
                     ],
              layout = go.Layout(title = rf"$Iteration: {i}$)", title_x=0.5),
               traces=[0, 1, 2, 3]))    

fig = make_subplots(
    rows=1, cols=3, column_widths = [0.3, 0.3, 0.3], subplot_titles=('Sample weight distribution', 'Accuracy of ensemble', 'Decisions boundaries'),
)

fig.add_traces(data=frames[0]["data"], rows=[1, 1, 1, 1], cols=[1, 2, 3, 3])
fig.update(frames = frames)
fig.update_yaxes(range=[0.4, 1.1], row=1, col=2)
fig.update_xaxes(range=[0, len(frames)], row=1, col=2)
fig.update_layout(updatemenus = [dict(type="buttons", buttons=[AnimationButtons.play(), 
                                                                   AnimationButtons.pause()])],  width=900, height = 300)
fig.show()